1. Name - Humera Bano
2. Github Username - humera811
3. USC ID - 7255691039

In [179]:
# organizing all imports together

import pandas as pd
import os
import glob
import numpy as np
import re
from sklearn.utils import resample


## __Time Series Classification__
Part 1: Feature Creation/Extraction
An interesting task in machine learning is classification of time series. In this problem,
we will classify the activities of humans based on time series obtained by a Wireless
Sensor Network.

#### __(a) Download the AReM data from: https://archive.ics.uci.edu/ml/datasets/ Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\ %29 . The dataset contains 7 folders that represent seven types of activities. In each folder, there are multiple files each of which represents an instant of a human performing an activity.1 Each file containis 6 time series collected from activities of the same person, which are called avg rss12, var rss12, avg rss13, var rss13, vg rss23, and ar rss23. There are 88 instances in the dataset, each of which contains 6 time series and each time series has 480 consecutive values.__

In [180]:
# removing extra comma in cycling dataset9.csv and cycling dataset14.csv manually as suggested on piazza
# bending2 dataset4.csv space separated (delimitter/sep_)
# https://www.geeksforgeeks.org/how-to-use-glob-function-to-find-files-recursively-in-python/

csv_files = glob.glob('../../data/AReM/*/*.csv', recursive=True)
# print(csv_files)
path = "../../data/AReM/"
actv_dir = ['bending1', 'bending2']
actv_dir2 = ['cycling', 'lying', 'sitting', 'standing', 'walking']
col = ['# Columns: time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']

#### __(b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data.__

In [181]:
# https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html#:~:text=Note%3A%20index_col%3DFalse%20can%20be,the%20end%20of%20each%20line.&text=Return%20a%20subset%20of%20the%20columns.
# https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

def all_act_label_df(trainset_testset_split, all_act_labels, start, end):
    for label in all_act_labels:
        for i in range(start,end+1):
            activity = str(i)
            if label=='bending2' and i==4: sep_=' '
            else:sep_=','
            filepath = '/dataset'+activity+'.csv'
            df = pd.read_csv(path+label+filepath, skiprows = 5, names=col,sep= sep_, index_col=False)
            # adding label piazza @500_f1
            df['label'] = label
            trainset_testset_split = pd.concat([df], ignore_index=True)
    return trainset_testset_split

#testset
testset = pd.DataFrame(columns=col)
testset = all_act_label_df(testset, actv_dir, 1, 2)
testset = all_act_label_df(testset, actv_dir2, 1, 3)

testset

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,label
0,0,24.75,5.76,13.25,3.03,18.75,1.48,walking
1,250,36.25,7.26,15.00,3.67,17.75,2.86,walking
2,500,37.00,3.94,20.25,1.30,16.25,1.64,walking
3,750,33.75,3.27,19.50,4.50,15.00,2.12,walking
4,1000,34.25,5.21,17.00,1.22,14.25,4.15,walking
...,...,...,...,...,...,...,...,...
475,118750,36.00,2.45,17.00,5.10,20.50,0.87,walking
476,119000,34.33,1.89,15.00,2.45,17.00,2.12,walking
477,119250,33.00,7.35,14.60,3.14,13.00,5.70,walking
478,119500,31.67,1.25,11.00,6.16,19.25,2.17,walking


In [182]:
#trainset
trainset = pd.DataFrame(columns=col)
trainset = all_act_label_df(trainset, actv_dir2, 4, 15)
trainset = all_act_label_df(trainset, actv_dir[:1], 3, 7)
trainset = all_act_label_df(trainset, actv_dir[1:2], 3, 6)

trainset

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,label
0,0,34.50,1.50,8.00,1.58,28.00,0.71,bending2
1,250,36.00,0.00,11.25,3.49,28.50,0.50,bending2
2,500,36.00,0.00,20.00,1.22,27.00,0.71,bending2
3,750,38.25,0.43,21.25,0.43,26.00,0.71,bending2
4,1000,37.50,1.12,22.00,0.82,23.67,2.49,bending2
...,...,...,...,...,...,...,...,...
475,118750,27.00,0.00,14.00,0.00,6.50,0.50,bending2
476,119000,27.00,0.00,15.00,0.00,4.00,1.00,bending2
477,119250,27.00,0.00,14.25,0.83,4.00,2.00,bending2
478,119500,27.00,0.00,13.33,1.25,1.50,0.50,bending2


### __(c) Feature Extraction Classification of time series usually needs extracting features from them.In this problem, we focus on time-domain features__


#### __i. Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc)__

Time-domain features are usually used in time series classification are:
1. Means
2. Standard Deviations
3. Skewness, Kurtosis
4. Maximum and minimum values

#### __ii. Extract the time-domain features minimum, maximum, mean, median, standard deviation, first quartile, and third quartile for all of the 6 time series in each instance.You are free to normalize/standardize features or use them directly__

In [202]:
# https://stackoverflow.com/questions/64227163/python-sort-list-by-number-in-file-name
# https://stackoverflow.com/questions/22809061/read-space-separated-data-with-pandas
def load_data(files, feature, orig_columns, segs=1, standard=False):
    #columns and labels
    # feature_list = ['min_', 'max_', 'mean_', 'median_', 'std_','first_quartile_', 'third_quartile_']
    # column_names = [ft + str(i) for i in range(1, 7) for ft in feature_list]
        # generate columns of describes and label with indecies
    describe_order = ['mean', 'std', 'min', '_1stquart_', 'median', '_3rdquart_','max']
    desc_order_idx = [desc + str(i) for i in range(1, 6 * segs + 1) for desc in describe_order]
    # stats_df = pd.DataFrame( columns = column_names)

# print(len(csv_files))

    for csvf in csv_files:
    # https://www.summet.com/dmsi/html/CSVdataFiles.html
        with open( csvf) as csv_file:
            allLines = csv_file.readlines()[5]
            csv_file.close
            sep = allLines [1]
            if sep == ' ' :
                feat_df = pd.read_csv(csvf, skiprows = 5, sep='\s+', header = None, error_bad_lines=False, warn_bad_lines=False)
            else:
                feat_df = pd.read_csv( csvf, skiprows=5, header = None, error_bad_lines=False, warn_bad_lines=False)
            
            feat_df.columns = ['# Columns: time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
        # dropping time and count column
            feat_df = feat_df.describe()
            feat_df = feat_df.drop(columns = '# Columns: time').drop('count')
        # https://pandas.pydata.org/pandas-docs/version/0.25.0/reference/api/pandas.DataFrame.T.html
            feat_df = feat_df.T[['min', 'max', 'mean', '50%', 'std', '25%', '75%']]
        # dataset of 88x42
            df_1 = pd.DataFrame(feat_df.to_numpy().reshape(1,42), columns = column_names)
            stats_df = stats_df.append(df_1, ignore_index=True)
        
# print(stats_df.isnull().any())
    return stats_df


,min1,max1,mean1,median1,standard deviation1,first quartile1,third quartile1,min2,max2,mean2,...,standard deviation5,first quartile5,third quartile5,min6,max6,mean6,median6,standard deviation6,first quartile6,third quartile6
0,36.25,48.00,43.969125,44.50,1.618364,43.3100,44.67,0.0,1.50,0.413125,...,3.318301,20.5000,23.7500,0.0,2.96,0.555312,0.490,0.487826,0.0000,0.8300
1,37.00,48.00,43.454958,43.25,1.386098,42.5000,45.00,0.0,1.58,0.378083,...,2.488862,22.2500,24.0000,0.0,5.26,0.679646,0.500,0.622534,0.4300,0.8700
2,33.00,47.75,42.179812,43.50,3.670666,39.1500,45.00,0.0,3.00,0.696042,...,3.849448,30.4575,36.3300,0.0,2.18,0.613521,0.500,0.524317,0.0000,1.0000
3,33.00,45.75,41.678063,41.75,2.243490,41.3300,42.75,0.0,2.83,0.535979,...,2.411026,28.4575,31.2500,0.0,1.79,0.383292,0.430,0.389164,0.0000,0.5000
4,37.25,45.00,40.624792,40.50,1.476967,39.2500,42.00,0.0,1.30,0.358604,...,2.188449,33.0000,36.0000,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,18.50,44.25,35.752354,36.00,4.614802,33.0000,39.33,0.0,12.60,3.328104,...,3.120057,14.0000,18.0625,0.0,9.39,3.069667,2.770,1.748326,1.7975,4.0600
84,24.25,45.00,37.177042,36.25,3.581301,34.5000,40.25,0.0,8.58,2.374208,...,2.890347,17.9500,21.7500,0.0,9.34,2.921729,2.500,1.852600,1.5000,3.9000
85,23.33,43.50,36.244083,36.75,3.822016,33.4575,39.25,0.0,9.71,2.736021,...,3.617702,15.7500,21.0000,0.0,11.15,3.530500,3.110,1.963685,2.1700,4.6175
86,26.25,44.25,36.957458,36.29,3.434863,34.5000,40.25,0.0,8.64,2.420083,...,2.851673,14.0000,18.2500,0.0,8.34,2.934625,2.525,1.631380,1.6600,4.0300


In [191]:
# reorder the columns
statistics = ['min','max','mean','median','std','_1stquart_', '_3rdquart_']
feature = [stat + str(i) for i in range(1, 7) for stat in statistics]

# load dataset as required
train_data = load_data(trainset, feature, col)
test_data = load_data(testset, feature, col)
train_data

,mean1,std1,min1,_1stquart_1,median1,_3rdquart_1,max1,mean2,std2,min2,...,median5,_3rdquart_5,max5,mean6,std6,min6,_1stquart_6,median6,_3rdquart_6,max6
0,36.25,48.00,43.969125,44.50,1.618364,43.3100,44.67,0.0,1.50,0.413125,...,3.318301,20.5000,23.7500,0.0,2.96,0.555312,0.490,0.487826,0.0000,0.8300
1,37.00,48.00,43.454958,43.25,1.386098,42.5000,45.00,0.0,1.58,0.378083,...,2.488862,22.2500,24.0000,0.0,5.26,0.679646,0.500,0.622534,0.4300,0.8700
2,33.00,47.75,42.179812,43.50,3.670666,39.1500,45.00,0.0,3.00,0.696042,...,3.849448,30.4575,36.3300,0.0,2.18,0.613521,0.500,0.524317,0.0000,1.0000
3,33.00,45.75,41.678063,41.75,2.243490,41.3300,42.75,0.0,2.83,0.535979,...,2.411026,28.4575,31.2500,0.0,1.79,0.383292,0.430,0.389164,0.0000,0.5000
4,37.25,45.00,40.624792,40.50,1.476967,39.2500,42.00,0.0,1.30,0.358604,...,2.188449,33.0000,36.0000,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,18.50,44.25,35.752354,36.00,4.614802,33.0000,39.33,0.0,12.60,3.328104,...,3.120057,14.0000,18.0625,0.0,9.39,3.069667,2.770,1.748326,1.7975,4.0600
84,24.25,45.00,37.177042,36.25,3.581301,34.5000,40.25,0.0,8.58,2.374208,...,2.890347,17.9500,21.7500,0.0,9.34,2.921729,2.500,1.852600,1.5000,3.9000
85,23.33,43.50,36.244083,36.75,3.822016,33.4575,39.25,0.0,9.71,2.736021,...,3.617702,15.7500,21.0000,0.0,11.15,3.530500,3.110,1.963685,2.1700,4.6175
86,26.25,44.25,36.957458,36.29,3.434863,34.5000,40.25,0.0,8.64,2.420083,...,2.851673,14.0000,18.2500,0.0,8.34,2.934625,2.525,1.631380,1.6600,4.0300


In [192]:
test_data

,mean1,std1,min1,_1stquart_1,median1,_3rdquart_1,max1,mean2,std2,min2,...,median5,_3rdquart_5,max5,mean6,std6,min6,_1stquart_6,median6,_3rdquart_6,max6
0,36.25,48.00,43.969125,44.50,1.618364,43.3100,44.67,0.0,1.50,0.413125,...,3.318301,20.5000,23.7500,0.0,2.96,0.555312,0.490,0.487826,0.0000,0.8300
1,37.00,48.00,43.454958,43.25,1.386098,42.5000,45.00,0.0,1.58,0.378083,...,2.488862,22.2500,24.0000,0.0,5.26,0.679646,0.500,0.622534,0.4300,0.8700
2,33.00,47.75,42.179812,43.50,3.670666,39.1500,45.00,0.0,3.00,0.696042,...,3.849448,30.4575,36.3300,0.0,2.18,0.613521,0.500,0.524317,0.0000,1.0000
3,33.00,45.75,41.678063,41.75,2.243490,41.3300,42.75,0.0,2.83,0.535979,...,2.411026,28.4575,31.2500,0.0,1.79,0.383292,0.430,0.389164,0.0000,0.5000
4,37.25,45.00,40.624792,40.50,1.476967,39.2500,42.00,0.0,1.30,0.358604,...,2.188449,33.0000,36.0000,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,18.50,44.25,35.752354,36.00,4.614802,33.0000,39.33,0.0,12.60,3.328104,...,3.120057,14.0000,18.0625,0.0,9.39,3.069667,2.770,1.748326,1.7975,4.0600
84,24.25,45.00,37.177042,36.25,3.581301,34.5000,40.25,0.0,8.58,2.374208,...,2.890347,17.9500,21.7500,0.0,9.34,2.921729,2.500,1.852600,1.5000,3.9000
85,23.33,43.50,36.244083,36.75,3.822016,33.4575,39.25,0.0,9.71,2.736021,...,3.617702,15.7500,21.0000,0.0,11.15,3.530500,3.110,1.963685,2.1700,4.6175
86,26.25,44.25,36.957458,36.29,3.434863,34.5000,40.25,0.0,8.64,2.420083,...,2.851673,14.0000,18.2500,0.0,8.34,2.934625,2.525,1.631380,1.6600,4.0300


### __iii. Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature.__

In [185]:
stat_std = stats_df.describe().loc['std']
stat_std

min_1               9.569975
max_1               4.394362
mean_1              5.335718
median_1            5.440054
std_1               1.772153
first_quartile_1    6.153590
third_quartile_1    5.138925
min_2               0.000000
max_2               5.062729
mean_2              1.574164
median_2            1.412244
std_2               0.884105
first_quartile_2    0.946386
third_quartile_2    2.125266
min_3               2.956462
max_3               4.875137
mean_3              4.008380
median_3            4.036396
std_3               0.946710
first_quartile_3    4.220658
third_quartile_3    4.171628
min_4               0.000000
max_4               2.183625
mean_4              1.166114
median_4            1.145586
std_4               0.458242
first_quartile_4    0.843620
third_quartile_4    1.552504
min_5               6.124001
max_5               5.741238
mean_5              5.675593
median_5            5.813782
std_5               1.024898
first_quartile_5    6.096465
third_quartile

In [186]:

bs_estimate = pd.DataFrame(columns = list(stats_df.columns))
for i in range(1000):
    
    # Taking sample with replacement
    bs_reps = resample(stats_df, replace = True, n_samples = stats_df.shape[0])
    
    # Calculat sd for the sample
    bs_estimate = bs_estimate.append(bs_reps.apply(np.std, ddof=1, axis = 0), ignore_index=True) 

# getting 90 percentile of data , which is nothing but 90% confidence
percentile_data = bs_estimate.apply(np.percentile, args=([5, 95],), axis = 0)
con_interval = pd.Series([(round(percentile_data .loc[0,col], 2),\
                           round(percentile_data .loc[1,col],2)) for col in list(stats_df.columns)], index=list(stats_df.columns))    
pd.DataFrame(list(zip(np.round(stat_std,2), con_interval)),
             index=stat_std.index, columns=['std', '90% Bootstrap CI'])

,std,90% Bootstrap CI
min_1,9.57,"(8.36, 10.74)"
max_1,4.39,"(3.28, 5.26)"
mean_1,5.34,"(4.74, 5.83)"
median_1,5.44,"(4.84, 5.96)"
std_1,1.77,"(1.57, 1.96)"
first_quartile_1,6.15,"(5.59, 6.63)"
third_quartile_1,5.14,"(4.36, 5.82)"
min_2,0.00,"(0.0, 0.0)"
max_2,5.06,"(4.66, 5.4)"
mean_2,1.57,"(1.41, 1.71)"


In [187]:
import warnings
from itertools import compress
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

warnings.filterwarnings("ignore")
data = get_features_split_n(normalize=True)
y_train = data['class']
x_train = data.drop(['class'], axis=1)
logreg = LogisticRegression()
rfe_res = RFE(estimator=logreg, n_features_to_select= 3)
rfe_res.fit(x_train, y_train)
best_features = list(compress(x_train.columns.tolist(),rfe_res.support_))
print(best_features)
#  use select_li to select 3 time domain features: [mean, std, min, first_quatile, median, third_quartile, max]
#  [1,0,0,0,0,1,1]
selected_features_list=[1,0,0,0,0,1,1]

['third_quartile3', 'min6', 'max6']
